## Carregue o Conjunto de Dados

In [ ]:
#Carregue o conjunto de dados do diretório fornecido
!git clone https://github.com/ByjusBrazil/PRO_1-4_C132_TextSentimentDataset

Cloning into 'PRO_1-4_C132_TextSentimentDataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 727.21 KiB | 5.86 MiB/s, done.


## Pandas:
O **Pandas** é uma biblioteca de código aberto construída sobre o Numpy e o Matplotlib. Ele fornece estruturas de dados de alto desempenho e fáceis de usar e ferramentas de análise de dados. 

Um DataFrame (quadro de dados) é uma estrutura de dados bidimensional, ou seja, os dados são alinhados de forma tabular em linhas e colunas da mesma forma que nosso conjunto de dados de teste. O DataFrame do Pandas consiste em três componentes principais, os dados, as linhas e as colunas.

O DataFrame do Pandas será criado carregando os conjuntos de dados de arquivos existentes do MS Excel, arquivos CSV ou banco de dados SQL. O DataFrame do Pandas também pode ser criado a partir de listas, dicionários, etc.



## Use o Pandas para exibir o conjunto de dados


In [ ]:
import pandas as pd

#leia o arquivo excel
train_data_raw = pd.read_excel("/content/PRO_1-4_C132_TextSentimentDataset/text-emotion-training-dataset.xlsx")

#exiba as primeiras cinco entradas do conjunto de dados de treinamento
train_data_raw.head()

,Text_Emotion
0,i didnt feel humiliated;sadness
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wro...
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy;anger


## Divida as linhas em duas colunas: Texto e Emoção

In [ ]:
# Divida as linhas em duas colunas: Texto e Emoção
train_data = pd.DataFrame(train_data_raw["Text_Emotion"].str.split(";",1).tolist(), 
                                                     columns = ['Texto','Emoção'])

In [ ]:
train_data.head()

,Texto,Emoção
0,"estou com a petronas há anos, sinto que a petr...",alegria
1,não me sinto seguro a ansiedade está de cada lado,alegria
2,tenho a sensação de que ela se divertiu e se e...,alegria
3,fico tonta por me sentir elegante em uma saia ...,alegria
4,eu preciso sentir a massa para ter certeza que...,alegria


## Dando etiquetas às emoções

In [ ]:
#Encontre emoções únicas
train_data["Emoção"].unique()

array(['alegria', 'amor', 'medo', 'raiva', 'surpresa', 'tristeza'],
      dtype=object)

In [ ]:
#Crie um dicionário para substituir emoções por etiquetas
encode_emotions = {"raiva": 0, "medo": 1, "alegria": 2, "amor": 3, "tristeza": 4, "surpresa": 5}

In [ ]:
train_data.replace(encode_emotions, inplace = True)
train_data.head()

,Texto,Emoção
0,"estou com a petronas há anos, sinto que a petr...",2
1,não me sinto seguro a ansiedade está de cada lado,2
2,tenho a sensação de que ela se divertiu e se e...,2
3,fico tonta por me sentir elegante em uma saia ...,2
4,eu preciso sentir a massa para ter certeza que...,2


## Converta o Dataframe em uma lista de conjunto de dados

In [ ]:
# Converta o Dataframe em uma lista de conjunto de dados

training_sentences = []
training_labels = []

# Anexe o texto e as emoções na lista usando o método 'loc'

for i in range(len(train_data)):
  
  sentence = train_data.loc[i, "Texto"]
  training_sentences.append(sentence)

  label = train_data.loc[i, "Emoção"]
  training_labels.append(label)

In [ ]:
# Verifique um texto e uma etiqueta aleatórios da lista

training_sentences[20], training_labels[20]


('tenho tomado ou miligramas ou vezes a quantidade recomendada e adormeci muito mais rápido, mas também me sinto muito engraçado',
 5)

## Tokenização e Preenchimento
O ato de converter texto em números é conhecido como **Tokenização**. A classe Tokenizer do Keras é usada para codificar entrada de texto em sequências inteiras.


In [ ]:
#importe o Tokenizer do tensorflow

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

#Defina os parâmetros do Tokenizer

vocab_size = 10000
embedding_dim = 16
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

#Crie um dicionário chamado word_index

word_index = tokenizer.word_index

#Verifique o valor numérico atribuído a uma palavra

word_index["o"]

49

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)

print(training_sequences[0])
print(training_sequences[1])
print(training_sequences[2])

[9, 15, 5, 30, 50, 51, 6, 3, 5, 30, 31, 10, 52, 53, 7, 31, 10, 54, 55]
[11, 2, 6, 56, 5, 57, 16, 8, 32, 58]
[12, 5, 59, 8, 3, 60, 17, 61, 7, 17, 62]


O **Preenchimento** é importante para fazer com que todas as frases contenham o mesmo número de palavras. Zeros são usados para preencher a sequência tokenizada para fazer o texto conter o mesmo número de tokens.

In [ ]:
#Defina os parâmetros das pad_sequences

from tensorflow.keras.preprocessing.sequence import pad_sequences

padding_type='post'
max_length = 100
trunc_type='post'


training_padded = pad_sequences(training_sequences, maxlen=max_length, 
                                padding=padding_type, truncating=trunc_type)

training_padded

array([[  9,  15,   5, ...,   0,   0,   0],
       [ 11,   2,   6, ...,   0,   0,   0],
       [ 12,   5,  59, ...,   0,   0,   0],
       ...,
       [  4,   6, 226, ...,   0,   0,   0],
       [  4,   2,   6, ...,   0,   0,   0],
       [  4,  11,   2, ...,   0,   0,   0]], dtype=int32)